In [1]:
"""
Same AR(1) model but using three different methods to illustrate all three optimizing techniques
"""
import numpy as np
import pandas as pd
import linkalman
import scipy
from linkalman.models import BaseConstantModel as BCM
from linkalman.core.utils import simulated_data, ft, df_to_list, list_to_df, \
        gen_PSD, get_ergodic, create_col, clean_matrix
from linkalman.core import Filter
import nlopt
%matplotlib inline 



In [2]:
def my_f(theta):
    """
    AR(1) model. In general, MLE is biased, so the focus should be 
    more on prediction fit, less on parameter estimation. The 
    formula here for Ar(1) is:
    y_t = c + Fy_{t-1} + epsilon_{t-1}
    """
    # Define theta
    phi_1 = 1 / (np.exp(theta[0])+1)
    sigma = np.exp(theta[1]) 
    sigma_R = np.exp(theta[2])
    # Generate F
    F = np.array([[phi_1]])
    # Generate Q
    Q = np.array([[sigma]]) 
    # Generate R
    R = np.array([[sigma_R]])
    # Generate H
    H = np.array([[1]])
    # Generate B
    B = np.array([[theta[3]]])
    # Collect system matrices
    M = {'F': F, 'Q': Q, 'H': H, 'R': R, 'B': B}

    return M


In [3]:
def my_solver(param, obj_func, **kwargs):
    """
    More complex solver function than the simple AR(1) case.
    The purpose is to provide an example of flexiblity of
    building solvers. Note I also suppress grad in nlopt_obj, 
    as linkalman uses only non-gradient optimizers
    """
    def nlopt_obj(x, grad, **kwargs):
        fval_opt = obj_func(x)
        if kwargs.get('verbose', False):
            print('fval: {}'.format(fval_opt))
        return fval_opt

    opt = nlopt.opt(nlopt.LN_BOBYQA, param.shape[0])
    obj = lambda x, grad: nlopt_obj(x, grad, **kwargs)
    opt.set_max_objective(obj)
    opt.set_xtol_rel(kwargs.get('xtol_rel', opt.get_xtol_rel()))
    opt.set_ftol_rel(kwargs.get('ftol_rel', opt.get_ftol_rel()))
    theta_opt = opt.optimize(param)
    fval_opt = opt.last_optimum_value()
    if kwargs.get('verbose_opt', False):
        print('fval: {}'.format(fval_opt))
        
    return theta_opt, fval_opt

In [4]:
# Initialize the model
x = 1
model = BCM()
model.set_f(my_f, x_0=x * np.ones([1, 1]))
model.set_solver(my_solver, xtol_rel=1e-4, verbose=True) 


In [5]:
# Generate fake data
theta = np.array([-1, -0.1, -0.2, 1])
T = 300

# Split train data
cutoff_t = np.floor(T * 0.7).astype(int)

# Generate missing data for forcasting
offset_t = np.floor(T * 0.9).astype(int)
x_col = ['const']
Xt = pd.DataFrame({x_col[0]: x * np.ones(T)})

df, y_col, xi_col = model.simulated_data(input_theta=theta, Xt=Xt, T=T)

# Create a training set
df_train = df.iloc[0:cutoff_t].copy()
df_test = df.copy()  # Keep the full data for forward prediction

# Create an offset
df_test['y_0_vis'] = df_test.y_0.copy()  # fully visible y
df_test.loc[df.index >= offset_t, ['y_0']] = np.nan

In [6]:
# Fit data using LLY:
theta_init = np.random.rand(len(theta))
model.fit(df_train, theta_init, y_col=y_col, x_col=x_col, 
              method='LLY')
theta_LLY = model.theta_opt

fval: -907.1845381817362
fval: -1016.424034931049
fval: -881.7250694361383
fval: -764.3123870067957
fval: -796.0824235024727
fval: -696.8809390016645
fval: -932.7581045790464
fval: -1137.3134729615222
fval: -1031.8390961017265
fval: -510.49091555964196
fval: -427.17871099537246
fval: -445.4836036098398
fval: -410.95317306354644
fval: -399.24658035121865
fval: -411.39789767545983
fval: -391.8433655901493
fval: -372.29036942997874
fval: -356.07292386816306
fval: -352.73205792675947
fval: -352.1284518422535
fval: -349.89026541556177
fval: -350.65054493972804
fval: -353.413299162927
fval: -349.4480068032395
fval: -349.30730660556554
fval: -349.29882849282836
fval: -349.6139814723312
fval: -349.87165969568514
fval: -349.36451132135255
fval: -349.1991153193835
fval: -349.22105027558314
fval: -349.1041622628345
fval: -349.01716840925866
fval: -348.9092279066723
fval: -348.8727795774758
fval: -348.94415918788593
fval: -349.061751338814
fval: -348.72100543345
fval: -348.63989092073757
fval: -34

In [7]:
# Fit data using both methods:
theta_init = np.random.rand(len(theta))
model.set_solver(my_solver, xtol_rel=1e-3, ftol_rel=1e-3, verbose_opt=True) 
model.fit(df_train, theta_init, y_col=y_col, x_col=x_col, method='EM', num_EM_iter=20)
model.set_solver(my_solver, xtol_rel=1e-4, verbose=True) 
model.fit(df_train, model.theta_opt, y_col=y_col, x_col=x_col, method='LLY')
theta_mix = model.theta_opt

fval: -578.8225692373226
fval: -463.9091518628262
fval: -407.7729163136786
fval: -367.1816050781739
fval: -362.76327940879594
fval: -357.6295015837937
fval: -346.3572055710148
fval: -328.18701916856025
fval: -309.65799794070915
fval: -303.9410125623983
fval: -300.6970140107255
fval: -298.29188082980124
fval: -296.95257233441515
fval: -296.1749648837872
fval: -295.64820728967663
fval: -295.25845845657255
fval: -294.94896058777385
fval: -294.6898522026315
fval: -294.44974127632383
fval: -294.13943122374155
fval: -316.51826698190627
fval: -441.63083101335906
fval: -317.89880842285106
fval: -322.95306741481926
fval: -637.962923022092
fval: -403.3051841736423
fval: -317.4990956535794
fval: -318.2820372665497
fval: -645.3964311430625
fval: -317.3813180802517
fval: -317.9560911383888
fval: -317.06227918026906
fval: -316.4290319278428
fval: -316.3631445036904
fval: -316.3734698988078
fval: -316.7518440803525
fval: -316.3166065823865
fval: -316.26252402938195
fval: -316.2584848848128
fval: -316

In [8]:
# Fit data using EM:
theta_init = np.random.rand(len(theta))
model.set_solver(my_solver, xtol_rel=1e-5, ftol_rel=1e-5, verbose_opt=True) 
model.fit(df_train, theta_init, y_col=y_col, x_col=x_col, EM_threshold=0.005, method='EM')
theta_EM = model.theta_opt

fval: -707.730638725526
fval: -581.195069731019
fval: -478.2593948942312
fval: -411.94477632830433
fval: -376.0808819174674
fval: -352.9841773415413
fval: -331.41088953869394
fval: -317.5800275547007
fval: -306.29856659768325
fval: -300.1836235198538
fval: -296.7152088189341
fval: -295.06858387260786
fval: -292.9582800608866
fval: -291.8763157623267
fval: -290.98043678372716
fval: -290.26900183267054
fval: -289.3624104723424
fval: -288.80411852805554
fval: -288.03548461966744
fval: -287.2894478847038
fval: -286.7928936961398
fval: -286.3185219313366
fval: -286.21175434596114
fval: -285.61948219186513
fval: -284.7883344139889
fval: -284.34028292298933
fval: -283.81102962032514
fval: -283.7542402075315
fval: -283.7376853678546
fval: -283.71204290700166
fval: -283.6839244767511
fval: -283.65311092504265
fval: -283.61989354746885
fval: -283.18950933249647
fval: -282.8730681146071
fval: -282.54964573002746
fval: -282.42282017020636
fval: -282.318194121396
fval: -282.2238068453831
fval: -282

In [9]:
# Make predictions from LLY:
df_LLY = model.predict(df_test, theta=theta_LLY)

# Make predictions from mixed models:
df_mix = model.predict(df_test, theta=theta_mix)

# Make predictions from EM:
df_EM = model.predict(df_test, theta=theta_EM)

# Make predictions using true theta:
df_true = model.predict(df_test, theta=theta)

In [10]:
# Calculate Statistics
RMSE = {}
RMSE['true'] = np.sqrt((df_true.y_0_filtered - df_true.y_0_vis).var())
RMSE['LLY'] = np.sqrt((df_LLY.y_0_filtered - df_LLY.y_0_vis).var())
RMSE['EM'] = np.sqrt((df_EM.y_0_filtered - df_EM.y_0_vis).var())
RMSE['mix'] = np.sqrt((df_mix.y_0_filtered - df_mix.y_0_vis).var())

M_error = {}
M_error['true'] = (df_true.y_0_filtered - df_true.y_0_vis).mean()
M_error['LLY'] = (df_LLY.y_0_filtered - df_LLY.y_0_vis).mean()
M_error['EM'] = (df_EM.y_0_filtered - df_EM.y_0_vis).mean()
M_error['mix'] = (df_mix.y_0_filtered - df_mix.y_0_vis).mean()
print(RMSE)
print(M_error)

{'true': 1.3313145136878208, 'LLY': 1.3253762455024292, 'EM': 1.3259338453361957, 'mix': 1.3253756889816979}
{'true': -0.07617562463886439, 'LLY': -0.0292044390201727, 'EM': -0.029088758798228913, 'mix': -0.029202154890296938}
